In [1]:
import sys
sys.path.append('..')

from keras.utils import np_utils
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import shutil
import importlib

%matplotlib inline
from sklearn.datasets import load_files
import pandas as pd
pd.set_option("display.max_colwidth", 75)

from keras.applications.imagenet_utils import preprocess_input

from util import paths_to_tensor

import tensorflow as tf
session_conf = tf.ConfigProto(
          device_count={'CPU': 1, 'GPU': 0},
          allow_soft_placement=True,
          log_device_placement=False
)
sess = tf.Session(config = session_conf)


#sys.path.append('mobilenet')
#sys.path.append('inceptionresnetv2')

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
# load TTD
ttd_basedir="/home/tutysara/src/myprojects/senti/dataset"
ttd_data_idx_path = ttd_basedir+ "/twitter_five_agrees.txt"
col_names = ["X", "y"]
ttd_data_df = pd.read_csv(ttd_data_idx_path, sep=" ", header=None, names=col_names)
ttd_data_df.X = ttd_data_df.X.apply(lambda x: ttd_basedir+"/Agg_AMT_Candidates/"+x)
print(ttd_data_df.shape)
ttd_data_df.head()

(882, 2)


,X,y
0,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/28800526...,0
1,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/33547480...,1
2,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/27176682...,1
3,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/10096570...,1
4,/home/tutysara/src/myprojects/senti/dataset/Agg_AMT_Candidates/13247920...,1


In [3]:
# work on twitter training dataset
# 3 agree, 4 agree, 5 agree

# report test accuracy on TTD
ttd_X1 = ttd_data_df.X.as_matrix()
ttd_X2 = preprocess_input(paths_to_tensor(ttd_X1))

In [4]:
def predict(model, n_samples=25):
    indices = np.random.permutation(ttd_X2.shape[0])
    indices_selected = indices[:n_samples]
    ttd_y_pred = model.predict(ttd_X2[indices_selected])  
    ttd_y_pred_two_classes=ttd_y_pred[:,[0,2]]
    ttd_y_true = ttd_data_df.y.as_matrix()[indices_selected]
    ttd_test_accuracy = 100*np.sum(np.argmax(ttd_y_pred_two_classes, axis=1)==ttd_y_true)/len(ttd_y_true)
    print('TTD Test accuracy: %.4f%%' % ttd_test_accuracy)

In [5]:
#n_samples=len(ttd_X1)
n_samples=15
print('n_samples', n_samples)

n_samples 15


In [6]:
from vgg19.vgg19_caffe_model import get_model
vgg_model = get_model(num_classes=3)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
vgg_model.load_weights("../vgg19/saved_models/fc_layers_caffe_vgg19_weights.hdf5")


In [8]:
%%timeit
predict(vgg_model, n_samples=n_samples)

TTD Test accuracy: 80.0000%
TTD Test accuracy: 73.3333%
TTD Test accuracy: 60.0000%
TTD Test accuracy: 86.6667%
TTD Test accuracy: 73.3333%
TTD Test accuracy: 60.0000%
TTD Test accuracy: 80.0000%
TTD Test accuracy: 66.6667%
10.4 s ± 14.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
vgg_model.load_weights("../vgg19/saved_models/all_layers_caffe_vgg19_weights.hdf5")

In [10]:
%%timeit
predict(vgg_model, n_samples=n_samples)

TTD Test accuracy: 73.3333%
TTD Test accuracy: 93.3333%
TTD Test accuracy: 73.3333%
TTD Test accuracy: 93.3333%
TTD Test accuracy: 53.3333%
TTD Test accuracy: 80.0000%
TTD Test accuracy: 80.0000%
TTD Test accuracy: 73.3333%
10.4 s ± 45.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
from mobilenet.mobilenet_model import get_model
mobilenet_model = get_model(num_classes=3)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
mobilenet_model.load_weights("../mobilenet/saved_models/fc_layers_mobilenet_weights.hdf5")


In [13]:
%%timeit
predict(mobilenet_model, n_samples=n_samples)

TTD Test accuracy: 66.6667%
TTD Test accuracy: 53.3333%
TTD Test accuracy: 86.6667%
TTD Test accuracy: 80.0000%
TTD Test accuracy: 100.0000%
TTD Test accuracy: 73.3333%
TTD Test accuracy: 53.3333%
TTD Test accuracy: 80.0000%
1.78 s ± 22.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
mobilenet_model.load_weights("../mobilenet/saved_models/all_layers_mobilenet_weights.hdf5")


In [15]:
%%timeit
predict(mobilenet_model, n_samples=n_samples)

TTD Test accuracy: 66.6667%
TTD Test accuracy: 73.3333%
TTD Test accuracy: 93.3333%
TTD Test accuracy: 66.6667%
TTD Test accuracy: 80.0000%
TTD Test accuracy: 73.3333%
TTD Test accuracy: 93.3333%
TTD Test accuracy: 73.3333%
1.78 s ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
from inceptionresnetv2.inceptionresnetv2_model import get_model
inceptionresntv2_model = get_model(num_classes=3)

Tensor("conv_7b_ac/Relu:0", shape=(?, 5, 5, 1536), dtype=float32)


In [17]:
inceptionresntv2_model.load_weights("../inceptionresnetv2/saved_models/fc_layers_inceptionresnetv2_weights.hdf5")


In [18]:
%%timeit
predict(inceptionresntv2_model, n_samples=n_samples)

TTD Test accuracy: 53.3333%
TTD Test accuracy: 60.0000%
TTD Test accuracy: 66.6667%
TTD Test accuracy: 80.0000%
TTD Test accuracy: 73.3333%
TTD Test accuracy: 73.3333%
TTD Test accuracy: 66.6667%
TTD Test accuracy: 53.3333%
5.25 s ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
inceptionresntv2_model.load_weights("../inceptionresnetv2/saved_models/all_layers_inceptionresnetv2_weights.hdf5")


In [20]:
%%timeit
predict(inceptionresntv2_model, n_samples=n_samples)

TTD Test accuracy: 66.6667%
TTD Test accuracy: 80.0000%
TTD Test accuracy: 46.6667%
TTD Test accuracy: 73.3333%
TTD Test accuracy: 60.0000%
TTD Test accuracy: 73.3333%
TTD Test accuracy: 73.3333%
TTD Test accuracy: 73.3333%
5.26 s ± 30 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
